# Getting Started with the Flow Super Agent using the AI Refinery SDK

Welcome! This notebook is a beginner-friendly guide to using the Accenture AI Refinery SDK to orchestrate a complex workflow with a **Flow Super Agent**.

This notebook uses the `DistillerClient` to manage a group of `UtilityAgent`s whose interactions are defined by a deterministic workflow. We will walk through setting up this multi-agent system to create a simple stock investment advisor.

## Step 1: Installation

First, we need to install the necessary libraries. We'll install `airefinery-sdk`, `python-dotenv` for managing credentials, `PyYAML` to create our configuration, and `mermaid-py` to render the workflow diagram.

You can run the cell below by selecting it and pressing **Shift + Enter**.

In [ ]:
%pip install airefinery-sdk python-dotenv PyYAML mermaid-py

## Step 2: Import Libraries

Now we import the necessary libraries for our notebook, including our `utils` module.

In [ ]:
import yaml
from IPython.display import display, Markdown
import utils

## Step 3: Create Agent Configuration and Visualize Workflow

The `DistillerClient` uses a YAML file to define all the agents and their workflows. The cell below will create the `flow_config.yaml` file for you. 

Immediately after, we'll use our utility function to read this file and render the workflow diagram.

In [ ]:
config_data = {
    'orchestrator': {
        'agent_list': [{'agent_name': 'Investment Strategy Advisor'}]
    },
    'utility_agents': [
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Price Researcher', 'agent_description': 'Search stock price movements.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Trend Researcher', 'agent_description': 'Research stock market trends.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Influence Researcher', 'agent_description': 'Search factors influencing stock prices.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Corporate Finance Researcher', 'agent_description': 'Research corporate finance and financial reports.'},
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Public Expectation Writer',
            'agent_description': 'Summarize public expectations based on stock trends.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'How do recent stock market trends reflect public sentiment and expectations for future price movements?', 'prompt': 'Analyze the latest stock market trends and explain how they influence public perception of future price changes.'},
                    {'question': 'What common themes emerge from stock trends that indicate investor confidence or concern?', 'prompt': 'Identify key patterns in stock trends that signal investor sentiment. Are there recurring indicators of optimism or fear?'},
                    {'question': 'Are there any major economic or geopolitical events influencing public expectations in the market?', 'prompt': 'List significant events (economic, political, or industry-specific) that have recently shaped public expectations in stock investments.'}
                ]
            }
        },
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Financial Report Writer',
            'agent_description': 'Write financial summaries based on corporate finance research.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'What are the key takeaways from the latest corporate financial reports, and how do they compare with industry benchmarks?', 'prompt': 'Summarize the most important points from the latest financial reports, including performance metrics and comparisons with industry standards.'},
                    {'question': 'How have recent earnings reports and balance sheet disclosures influenced investor sentiment?', 'prompt': 'Explain how recent earnings reports and balance sheets have impacted investor confidence, referencing key financial indicators.'},
                    {'question': 'Are there any financial indicators or metrics that stand out in the companies under research?', 'prompt': 'Highlight notable financial metrics from corporate reports that could be useful in making investment decisions.'}
                ]
            }
        },
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Investment Suggestion Writer',
            'agent_description': 'Generate insights based on stock research and financial reports.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'Which stock or company is being analyzed for investment considerations?', 'prompt': 'Identify the stock or company mentioned in the conversation history. Provide context on why it is being analyzed, referencing relevant past discussions or queries.'},
                    {'question': 'What are the key insights from stock trends, financial reports, and market sentiment?', 'prompt': 'Analyze stock price movements, market trends, corporate financial reports, and investor sentiment. Identify significant patterns and factors affecting the stock’s performance.'},
                    {'question': 'Based on the analysis, should investors consider buying, selling, or holding this stock?', 'prompt': 'Evaluate the stock\'s current valuation, market trends, financial stability, and risk factors. Provide a single, definitive investment decision on whether investors should buy, sell, or hold.'}
                ]
            }
        }
    ],
    'super_agents': [
        {
            'agent_class': 'FlowSuperAgent',
            'agent_name': 'Investment Strategy Advisor',
            'agent_description': 'Provides investment insights based on stock and finance research.',
            'config': {
                'goal': 'Generate investment recommendations based on stock research, trends, financial reports, and public expectations.',
                'agent_list': [
                    {'agent_name': 'Stock Price Researcher', 'next_step': ['Stock Trend Researcher', 'Stock Influence Researcher', 'Financial Report Writer']},
                    {'agent_name': 'Stock Trend Researcher', 'next_step': ['Public Expectation Writer']},
                    {'agent_name': 'Stock Influence Researcher', 'next_step': ['Investment Suggestion Writer']},
                    {'agent_name': 'Corporate Finance Researcher', 'next_step': ['Financial Report Writer']},
                    {'agent_name': 'Financial Report Writer', 'next_step': ['Investment Suggestion Writer', 'Stock Influence Researcher']},
                    {'agent_name': 'Public Expectation Writer', 'next_step': ['Investment Suggestion Writer']},
                    {'agent_name': 'Investment Suggestion Writer'}
                ]
            }
        }
    ]
}

config_file_path = 'flow_config.yaml'
with open(config_file_path, 'w') as f:
    yaml.dump(config_data, f, default_flow_style=False)

print(f"'{config_file_path}' created successfully.")

# Visualize the workflow using our utility function
diagram = utils.display_workflow_diagram(config_data)
display(diagram)

## Step 4: Login and Initialize the Distiller Client

Our utility library handles the login and client setup. This cell calls our functions to perform the authentication and create the `DistillerClient` instance.

In [ ]:
# Securely log in using our utility function
utils.secure_login()

# Create a distiller client
project_name = "stock_invest_advisor"
distiller_client = utils.initialize_client(config_path=config_file_path, project_name=project_name)

## Step 5: Ask for Investment Advice

Now that the client is initialized, we can ask our first question. The response from the agent will be streamed back. We will format each message to show which internal agent (the 'role') generated the content, similar to how the OpenAI API structures chat responses.

In [ ]:
# Use the client to ask a question to the Flow Super Agent
async with distiller_client(
    project=project_name,
    uuid="test_user_flow", # A unique identifier for the user/session
) as dc:
    print("\nSending request to the Flow Super Agent... (This may take a moment)")
    responses = await dc.query(
        query="Should I invest in NVDA this quarter?"
    )
    
    print("\n--- Agents Response ---")
    async for response in responses:
        # The response object from the SDK likely contains the agent's name.
        # We'll use this to represent the 'role'.
        role = response.get("role", "system")
        content = response.get("content", "")

        # Display the role and then the content in a structured way.
        print(f"🗣️ Role: {role}")
        display(Markdown(content))
        print("-------------------\n")

## Step 6: Try Your Own Questions!

Now it's your turn. You can modify the `my_question` in the cell below to ask about a different stock. The output will again be formatted to show the chat roles.

In [ ]:
# --- Try your own question here! ---
my_question = "Should I invest in MSFT this quarter?"
# ------------------------------------------------

async with distiller_client(project=project_name, uuid="test_user_flow_2") as dc:
    print(f"\nAsking the agent: '{my_question}'")
    responses = await dc.query(
        query=my_question,
    )
    print("\n--- Agents Response ---")
    async for response in responses:
        # The response object from the SDK likely contains the agent's name.
        # We'll use this to represent the 'role'.
        role = response.get("role", "system")
        content = response.get("content", "")

        # Display the role and then the content in a structured way.
        print(f"🗣️ Role: {role}")
        display(Markdown(content))
        print("-------------------\n")

## Congratulations!

You've successfully used the `FlowSuperAgent` to manage a complex, multi-agent workflow and can now see the individual contributions from each agent in the flow.